In [ ]:
%pip install nbformat
%pip install imbalanced-learn
%pip install catboost
%pip install imblearn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import nbformat

# Read your notebook (assuming version 4 for example purposes)
nb = nbformat.read("ml_project.ipynb", as_version=4)

# Normalize the notebook to add missing id fields and other updates
nbformat.validator.validate(nb)

# Write the normalized notebook back to a file
nbformat.write(nb, "ml_project_normalized.ipynb")

%run ml_project_normalized.ipynb

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
dropped columns
cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned breed
cleaned color
dropped columns


/tmp/ipykernel_1804608/850107665.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt_series = pd.to_datetime(df['intake_time'], errors='coerce')


cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned breed
cleaned color
Done running ml_project.ipynb.


In [46]:
from imblearn.over_sampling import SMOTENC
from imblearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.metrics import make_scorer
from sklearn.utils.class_weight import compute_class_weight
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np

def train_classifier(X_train, y_train, X_test, rare_classes, categorical_features):
  """
  Trains a CatBoost model using SMOTE for class balancing and hyperparameter tuning.
  CatBoost handles categorical features natively.

  Parameters:
    X_train (pd.DataFrame): Training features.
    y_train (pd.Series or np.array): Training target values.
    X_test (pd.DataFrame): Test features.
    rare_classes (list): List of integer-encoded classes to be oversampled with SMOTE.
    categorical_features (list): List of column names for categorical features.

  Returns:
    best_estimator: The best estimator from RandomizedSearchCV.
    test_predictions: The predicted labels for X_test from the best estimator.
  """

  # Compute rare class sampling target
  y_series        = pd.Series(y_train)
  max_count       = y_series.value_counts().max()
  sampling_target = {cls: max_count for cls in rare_classes}

  # Convert categorical column names to indices for SMOTENC
  cat_feature_indices = [X_train.columns.get_loc(col) for col in categorical_features]

  # Assumes y_train is label-encoded (0 to 4)
  class_labels = np.unique(y_train)

  class_weights = compute_class_weight(
      class_weight='balanced',
      classes=class_labels,
      y=y_train
  )

  print("Class weights:", class_weights)

  # Define pipeline (SMOTENC is instantiated inside to avoid data leakage)
  pipeline = Pipeline([
    ('smote', SMOTENC(
        categorical_features=cat_feature_indices,
        sampling_strategy=sampling_target,
        random_state=42
    )),
    ('clf', CatBoostClassifier(
        verbose=0,
        cat_features=categorical_features,
        random_state=42,
        thread_count=3,  # Limit threads to avoid resource exhaustion
        class_weights=class_weights
    ))
  ])

  # Define hyperparameter search space
  param_dist = {
    'clf__iterations': [100, 300, 500],
    'clf__depth': [4, 6, 8, 10],
    'clf__learning_rate': [0.01, 0.05, 0.1],
    'clf__l2_leaf_reg': [1, 3, 5, 7]
  }

  # Randomized search CV
  search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=5,
    cv=5,
    scoring='accuracy',
    verbose=3,
    random_state=42,
    n_jobs=2  # Limit parallelism
  )

  print(f"\n[INFO] Starting training with {len(X_train)} samples, {len(y_train)} labels")
  # Fit the model
  search.fit(X_train, y_train)
  print(f"[INFO] Training complete. Best model fitted on {len(X_train)} samples.\n")


  # Evaluation
  print('Best parameters:', search.best_params_)
  print('Best cross-validation accuracy:', search.best_score_)

  # Optional generalization check with custom scorer
  try:
    cv_scores = cross_val_score(
      search.best_estimator_,
      X_train,
      y_train,
      cv=5,
      verbose=3,
      scoring=make_scorer(classification_report_with_accuracy_score)
    )
    print('Generalization accuracy (via cross_val_score):', cv_scores.mean())
  except Exception as e:
    print(f"Cross-validation scoring failed: {e}")

  # Predict on test set
  test_predictions = search.predict(X_test)

  return search.best_estimator_, test_predictions


In [47]:
import pandas as pd 
from sklearn.preprocessing import LabelEncoder

X_train = df_train.drop(columns=['outcome_type'])
y_train = df_train['outcome_type']
X_test = df_test

# label coder will only be used for the outcome labels
le = LabelEncoder()
y_train = le.fit_transform(y_train)
print('Encoding mapping:', le.classes_)

# Define rare classes that need oversampling 
rare_classes = [
  label for label, count in pd.Series(y_train).value_counts().items()
  if count < 0.05 * len(y_train)
]
print("Rare classes:")
for cls in rare_classes:
  print(f"  {cls}: {le.classes_[cls]}")

# Define categorical features - either column names or column indices
cat_cols = {'intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake', 'breed', 'primary_color'}
categorical_features = [col for col in X_train.columns if col in cat_cols]

# Train model and get test predictions
best_model, test_predictions = train_classifier(
  X_train=X_train,
  y_train=y_train,
  X_test=X_test,
  rare_classes=rare_classes,
  categorical_features=categorical_features
)

# Decode integer predictions back to string labels
decoded_preds = le.inverse_transform(test_predictions)

# Save predictions using your utility function
save_predictions(decoded_preds, model_name='catboost')


Encoding mapping: ['Adoption' 'Died' 'Euthanasia' 'Return to Owner' 'Transfer']
Rare classes:
  2: Euthanasia
  1: Died
Class weights: [ 0.4038769  21.35542747  6.44563642  1.33945894  0.63473618]

[INFO] Starting training with 111155 samples, 111155 labels
Fitting 5 folds for each of 5 candidates, totalling 25 fits


KeyboardInterrupt: 

In [ ]:
# print(df_train.columns)